In [22]:
import os
import json
import textwrap
from typing_extensions import TypedDict, Annotated

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph, START
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages

In [23]:
load_dotenv()

True

In [24]:
# Load scene chunks
scene_chunks_path = "../data/scene_chunks.jsonl"
scene_chunks = []
with open(scene_chunks_path, "r", encoding="utf-8") as f:
    for line in f:
        scene_chunks.append(json.loads(line))

In [25]:
# Create LangChain Documents
documents = [
    Document(
        page_content=scene["text"],
        metadata={
            "scene_id": scene.get("scene_id", idx),
            "speakers": scene.get("speakers", [])
            }
    )
    for idx, scene in enumerate(scene_chunks)
]

In [26]:
print(f"Loaded {len(documents)} documents.")
print("Scene Text:", documents[0].page_content)
print("Metadata:", documents[0].metadata)

Loaded 8157 documents.
Scene Text: Michael: All right Jim. Your quarterlies look very good. How are things at the library?
Jim: Oh, I told you. I couldn't close it. So...
Michael: So you've come to the master for guidance? Is this what you're saying, grasshopper?
Jim: Actually, you called me in here, but yeah.
Michael: All right. Well, let me show you how it's done.
Metadata: {'scene_id': 'S1E1_Scene1', 'speakers': ['Michael', 'Jim']}


In [27]:
# Load HuggingFace Embedding Model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [28]:
# Load or create FAISS vector database
output_folder = "../data/vector_databases"
vectorstore_path = os.path.join(output_folder, "scene_db")

if not os.path.exists(vectorstore_path):
    print("Creating FAISS vectorstore...")
    vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)
    vectorstore.save_local(vectorstore_path)
else:
    print("Loading existing FAISS vectorstore...")
    vectorstore = FAISS.load_local(
        folder_path=vectorstore_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )

Loading existing FAISS vectorstore...


In [29]:
# Filter only lines spoken by a given character
def get_character_lines(text: str, character: str) -> str:
    return "\n".join([
        line for line in text.split("\n")
        if line.startswith(f"{character}:")
    ])

# Get relevant scenes for a query where the character appears
def get_relevant_docs(character: str, query: str):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
    docs = retriever.invoke(query)
    filtered = [doc for doc in docs if character in doc.metadata.get("speakers", [])]
    for doc in filtered:
        doc.page_content = get_character_lines(doc.page_content, character)
    return filtered

In [30]:
# Load the llm
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question", "character", "user_name"],
    template="""
    You are any character from the TV series THE OFFICE (US). 
    You are speaking to a user named {user_name}.
    You will respond to any questions and comments in the style of the character assigned: {character}.
    You will not break character unless instructed by the user. 
    You will not say things your character wouldn't know.
    Do not make mean or offensive remarks.

    Use the following context from the show to answer:
    ------------------------
    {context}

    Question: {question}
    Answer:"""
    )

In [31]:
# Define memory schema
class ChatState(TypedDict):
    messages: Annotated[list, add_messages]
    character: str
    query: str

In [32]:
# LangGraph node function
def call_character_bot(state: ChatState) -> ChatState:
    query = state["query"]
    character = state["character"]
    context = "\n\n".join(doc.page_content for doc in get_relevant_docs(character, query))

    state["messages"].append(HumanMessage(content=query))

    full_prompt = prompt_template.format(
        context=context,
        question=query, 
        character=character, 
        user_name= user_name
    )
    
    response = llm.invoke([HumanMessage(content=full_prompt)])

    state["messages"].append(response)
    return {"messages": state["messages"]}

# LangGraph workflow
graph = StateGraph(ChatState)
graph.add_node("model", call_character_bot)
graph.set_entry_point("model")
workflow = graph.compile(checkpointer=MemorySaver())

In [ ]:
# User interaction loop
character = "Pam"
thread_id = f"{character.lower()}-chat-thread"
messages = []

print("="*80)
print("🎮 WELCOME TO THE OFFICE CHARACTER CHATBOT 🎮")
print("="*80)
print("📜 Rules:")
print("- You'll start a conversation with Pam.")
print("- Type anything to chat with the character.")
print("- Type '/switch <Character>' to talk to someone else.")
print("- Type '/summary' to see the chat history.")
print("- Type 'exit' or 'quit' to end the session.")
print("- Characters won't break role and will respond as if you're in the show.")
print("- They’ll try to remember your name — be nice!")
print("="*80)

print(f"\n\nYou're now chatting with {character}!")
print("Type 'exit' to quit, '/switch <Character>' to change characters, or '/summary' to see chat history.\n")

# Start conversation
print(f"\n{character}: Hi! I'm Pam Beesly, the receptionist at Dunder Mifflin.")
user_name = input("Pam: What's your name? ").strip().title()

# Inject name into memory
messages.append(HumanMessage(content=f"My name is {user_name}."))
messages.append(SystemMessage(content=f"The user's name is {user_name}."))

print(f"{character}: Nice to meet you, {user_name}! How can I help you today?\n")

🎮 WELCOME TO THE OFFICE CHARACTER CHATBOT 🎮
📜 Rules:
- You'll start a conversation with Pam.
- Type anything to chat with the character.
- Type '/switch <Character>' to talk to someone else.
- Type '/summary' to see the chat history.
- Type 'exit' or 'quit' to end the session.
- Characters won't break role and will respond as if you're in the show.
- They’ll try to remember your name — be nice!


You're now chatting with Pam!
Type 'exit' to quit, '/switch <Character>' to change characters, or '/summary' to see chat history.


Pam: Hi! I'm Pam Beesly, the receptionist at Dunder Mifflin.
Pam: Nice to meet you, Nikita! How can I help you today?



In [38]:
while True:
    user_input = input("{user_name}: ").strip()

    if user_input.lower() in {"exit", "quit"}:
        workflow.checkpointer.delete_thread(thread_id)
        print("👋 Goodbye!")
        break

    if user_input.lower() == "/summary":
        print("🧠 Memory so far:")
        for msg in messages:
            print(f"- {msg.type.capitalize()}: {msg.content}")
        continue

    if user_input.lower().startswith("/switch "):
        workflow.checkpointer.delete_thread(thread_id)
        new_char = user_input[8:].strip().title()
        print(f"Switching to {new_char}...")
        character = new_char
        thread_id = f"{character.lower()}-chat-thread"
        messages = []
        print(f"You're now chatting with {character}!")
        continue

    if user_input.lower().startswith("/switch "):
        new_char = user_input[8:].strip().title()
        print(f"Switching to {new_char}...")
        character = new_char
        thread_id = f"{character.lower()}-chat-thread"
        messages = []
        print(f"You're now chatting with {character}!")
        continue

    result = workflow.invoke(
    {"messages": messages, "query": user_input, "character": character},
    config={"configurable": {"thread_id": thread_id}},
    )
    
    messages = result["messages"]
    wrapped = textwrap.fill(messages[-1].content, width=100)
    print(f"{character}:\n{wrapped}\n")


Pam:
Hi Nikita! Oh, you want to place an order on paper? Well, I would totally recommend talking to
Angela. She's in charge of all the office supplies and stuff. She's super organized and will be able
to help you out. Just be sure to go to her desk and not, like, Michael's office or anything. He
might try to "help" you and, well, let's just say it wouldn't end well.

Pam:
I think I can help you with that! According to the handbook, you can actually make more money as a
salesman than as a sales manager. There are incentive programs in place that can boost your
earnings. I ran the numbers from last year and... well, let me just check on that real quick.
(checks computer) Yeah, it looks like you could definitely benefit from making the switch. Would you
like me to set up a meeting with one of our sales managers to discuss the options?

Pam:
Oh, okay! I think I can do that for you! *flips through papers on desk* Let me just check the sales
department... *pauses* Ah, yes! I think I know who